# Setup

In [130]:
import os, yaml, sqlite3, hashlib, random, time
from datetime import datetime, timezone

import pandas as pd
import numpy as np

from utils import extract_license_plate_number

# Load and extract configuration variables
# ----
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
# Path where license plate images of arriving vehicles are saved
arrivals_path = config['file_paths']['arrivals']
# Path where license plate images of departing vehicles are saved
departures_path = config['file_paths']['departures']
# Path to sample images, selection of 50 from https://www.kaggle.com/datasets/nickyazdani/license-plate-text-recognition-dataset
images_path = config['file_paths']['images']
# Path to the sqlite database
db_path = os.path.join(config['database']['location'], config['database']['name']+'.db')

# Start fresh instead of appending to existing database tables
FRESH_START = True
# Number of days to generate sample data for
N_DAYS = 7
# Maximum visit length in hours
MAX_VISIT_LEN_HRS = 4

## Database & Tables

In [128]:
# Connect to the database and create tables if they do not exist
# ----
# Note - if the database does not exist it will be created
with sqlite3.connect(db_path) as db_conn:
    db_cur = db_conn.cursor()  # cursor to execute SQL statements on db_conn
    
    # Create tables if they do not already exist
    ## Arrivals
    if FRESH_START:
        res = db_cur.execute('drop table arrivals')
    res = db_cur.execute(
    """create table if not exists arrivals(
        arrival_record_id string, license_plate_number string, arrival_ts_utc timestamp, arrival_comment string, arrival_created_at_utc timestamp
    )""")
    ## Departures
    if FRESH_START:
        res = db_cur.execute('drop table departures')  # uncomment to delete table and start fresh
    res = db_cur.execute(
    """create table if not exists departures(
        arrival_record_id string, license_plate_number string, departure_ts_utc timestamp, departure_comment string, departure_created_at_utc timestamp
    )""")

## Generate Sample Data

In [148]:
images = os.listdir(images_path)
n_daily_visitors = np.random.normal(20, 5, N_DAYS).astype(int)
daily_visitors = [np.random.choice(images, v, replace=False) for v in n_daily_visitors]

for i in range(N_DAYS):
    date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=i+1)
    arrival_times = [
        date + timedelta(
            hours=random.randint(0,23),
            minutes=random.randint(0,59),
            seconds=random.randint(0,59)
        )
        for j in range(len(daily_visitors[i]))
    ]
    departure_times = [
        arrival_times[j] + timedelta(
            hours=random.randint(0, MAX_VISIT_LEN_HRS),
            minutes=random.randint(0,59),
            seconds=random.randint(0,59)
        )
        for j in range(len(daily_visitors[i]))
    ]

    for j in range(len(daily_visitors[i])):
        car = daily_visitors[i][j]
        arrival_creation_time = arrival_times[j]
        departure_creation_time = departure_times[j]

        base_file_path = os.path.join(images_path, car)
        car = os.path.splitext(car)
        arrival_file_path = os.path.join(
            arrivals_path,
            'staging',
            f"{car[0]}_{datetime.strftime(arrival_creation_time, '%Y%m%dT%H%M%S')}"+car[1]
            
        )
        departure_file_path = os.path.join(
            departures_path,
            'staging',
            f"{car[0]}_{datetime.strftime(departure_creation_time, '%Y%m%dT%H%M%S')}"+car[1]
        )
        os.system(f'cp {base_file_path} {arrival_file_path}')
        os.utime(arrival_file_path, (int(time.mktime(arrival_creation_time.timetuple())), int(time.mktime(arrival_creation_time.timetuple()))))
        os.system(f'cp {base_file_path} {departure_file_path}')
        os.utime(departure_file_path, (int(time.mktime(departure_creation_time.timetuple())), int(time.mktime(departure_creation_time.timetuple()))))

# Arrivals

In [150]:
# License plate images from new arrivals written to staging sub-folder
new_arrivals = os.listdir(os.path.join(arrivals_path, 'staging'))
# If there are new arrivals in the staging folder, add them to the database
if len(new_arrivals) > 0:
    # TO DO - add batch processing option for scalable backfilling
    for arrival_image in new_arrivals:
        # ignore non-JPG files
        if arrival_image.endswith('.jpg'):
            # Extract license plate number from image using PyTorch model
            image_path = os.path.join(arrivals_path, 'staging', arrival_image)
            license_plate_number = extract_license_plate_number(image_path)
            # Get arrival timestamp - assumed equal to the last modified timestamp of the image file
            arrival_ts_utc = datetime.strftime(datetime.fromtimestamp(os.path.getmtime(image_path), timezone.utc), '%Y-%m-%d %H:%M:%S.%f')
            # Get timestamp the arrival record was created at (now)
            arrival_created_at_utc = datetime.strftime(datetime.utcnow(), '%Y-%m-%d %H:%M:%S.%f')
            # Create arrival record ID by hasing the combination of the license plate number and the arrival timestamp
            arrival_record_id = hashlib.md5(f'{license_plate_number};{arrival_ts_utc}'.encode('utf-8')).hexdigest()

            with sqlite3.connect(db_path) as db_conn:
                # Write the record to the arrivals database
                db_cur = db_conn.cursor()
                res = db_cur.execute(
                f"""
                insert into arrivals (arrival_record_id, license_plate_number, arrival_ts_utc, arrival_comment, arrival_created_at_utc)
                values('{arrival_record_id}', '{license_plate_number}', '{arrival_ts_utc}', '{arrival_image}', '{arrival_created_at_utc}')
                """
                )
                # Check whether write process was successful
                res = db_cur.execute(f"select * from arrivals where arrival_record_id = '{arrival_record_id}'").fetchall()
                if len(res) > 0:
                    print('Arrival Record Inserted - OK')
                    print(res)
                    # If successful, move processed image from staging to production
                    os.rename(
                        os.path.join(arrivals_path, 'staging', arrival_image),
                        os.path.join(arrivals_path, 'production', arrival_image)
                    )
                else:
                    print('Arrival Record Inserted - FAIL')
                    print(arrival_image)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Arrival Record Inserted - OK
[('627ffb0901a3d73e68b489c948e66e36', 'RT:7178', '2023-12-29 00:09:50.000000', '28_20231228T190950.jpg', '2023-12-29 22:29:27.483622')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Arrival Record Inserted - OK
[('414f2a5b344b31bf6c079ad421f26084', '8440 GZ', '2023-12-22 23:39:10.000000', '27_20231222T183910.jpg', '2023-12-29 22:29:34.560475')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


...

Arrival Record Inserted - OK
[('0509b3eb87bd0ef6adcb7548c6ee7b07', '5807AF', '2023-12-27 21:14:26.000000', '26_20231227T161426.jpg', '2023-12-29 22:45:01.666834')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Arrival Record Inserted - OK
[('cdd8ea26f7a60bde3a9de21f49de5daf', '8440 GZ', '2023-12-24 12:48:11.000000', '27_20231224T074811.jpg', '2023-12-29 22:45:08.596639')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Arrival Record Inserted - OK
[('c58735d35da3b32355b5a342ca00ae8f', 565481, '2023-12-27 15:49:53.000000', '9_20231227T104953.jpg', '2023-12-29 22:45:15.288557')]


# Departures

In [151]:
# License plate images from new departures written to staging sub-folder
new_departures = os.listdir(os.path.join(departures_path, 'staging'))
# If there are new departures in the staging folder, add them to the database
if len(new_departures) > 0:
    # TO DO - add batch processing option for scalable backfilling
    for departure_image in new_departures:
        # ignore non-JPG files
        if departure_image.endswith('.jpg'):
            # Extract license plate number from image using PyTorch model
            image_path = os.path.join(departures_path, 'staging', departure_image)
            license_plate_number = extract_license_plate_number(image_path)
            # Get departure timestamp - assumed equal to the creation timestamp of the image file
            departure_ts_utc = datetime.strftime(datetime.fromtimestamp(os.path.getmtime(image_path), timezone.utc), '%Y-%m-%d %H:%M:%S.%f')
            # Get timestamp the departure record was created at (now)
            departure_created_at_utc = datetime.strftime(datetime.utcnow(), '%Y-%m-%d %H:%M:%S.%f')
            with sqlite3.connect(db_path) as db_conn:
                db_cur = db_conn.cursor()

                # Get the last recorded arrival_record_id prior to the creation of the departure image
                # TO DO - update to handle edge cases where arrival records are missing or delayed due to camera or network issues
                arrival_record_id = db_cur.execute(
                f"""
                select arrival_record_id, arrival_ts_utc
                from arrivals
                where license_plate_number = '{license_plate_number}'
                    and arrival_ts_utc < '{departure_ts_utc}'
                order by arrival_ts_utc desc
                """
                ).fetchone()[0]

                # Write the record to the departures database
                res = db_cur.execute(
                f"""
                insert into departures (arrival_record_id, license_plate_number, departure_ts_utc, departure_comment, departure_created_at_utc)
                values('{arrival_record_id}', '{license_plate_number}', '{departure_ts_utc}', '{departure_image}', '{departure_created_at_utc}')
                """
                )
                # Check whether write process was successful
                res = db_cur.execute(f"select * from departures where arrival_record_id = '{arrival_record_id}'").fetchall()
                if len(res) > 0:
                    print('Departure Record Inserted - OK')
                    print(res)
                    # If successful, move processed image from staging to production
                    os.rename(
                        os.path.join(departures_path, 'staging', departure_image),
                        os.path.join(departures_path, 'production', departure_image)
                    )
                else:
                    print('Departure Record Inserted - FAIL')
                    print(departure_image)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Departure Record Inserted - OK
[('3f1fec3e41554f23c46367cf82b9e3ec', 'RZ 0047', '2023-12-23 15:10:27.000000', '1_20231223T101027.jpg', '2023-12-29 22:45:22.175727')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Departure Record Inserted - OK
[('942f9cf713d766fabc1179c518b1ffcc', 'MY5459', '2023-12-26 16:15:33.000000', '18_20231226T111533.jpg', '2023-12-29 22:45:29.130276')]


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


...

Departure Record Inserted - OK
[('614f79b98ee9f3315f00cf4b737088fa', 'RC.0038', '2023-12-26 09:06:30.000000', '66_20231226T040630.jpg', '2023-12-29 23:00:55.129144')]


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Departure Record Inserted - OK
[('a859f37ae8c7811b598840552851904b', 'RT:7178', '2023-12-26 04:55:47.000000', '28_20231225T235547.jpg', '2023-12-29 23:01:02.238634')]


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Departure Record Inserted - OK
[('71b92b1f632161085232596014b42ad2', 'MY5459', '2023-12-24 01:53:27.000000', '18_20231223T205327.jpg', '2023-12-29 23:01:09.137905')]


# Analysis Example

In [156]:
with sqlite3.connect(db_path) as db_conn:
    db_cur = db_conn.cursor()
    df = pd.DataFrame(db_cur.execute("""
    select date(arrival_ts_utc) as date,
        count(*) as total_visitors,
        count(distinct license_plate_number) as unq_visitors,
        sum(visit_duration_minutes)/count(*) as avg_visit_length_minutes,
        count(*)*1.0/count(distinct license_plate_number) as avg_visits_per_unq_visitor
    from (
        select a.arrival_record_id,
            a.license_plate_number,
            a.arrival_ts_utc,
            b.departure_ts_utc,
            (julianday(departure_ts_utc) - julianday(arrival_ts_utc))*24*60 as visit_duration_minutes
        from arrivals a
            left join departures b on a.arrival_record_id = b.arrival_record_id
    )
    group by 1
    """).fetchall(),columns=['date', 'total_visitors', 'unq_visitors', 'avg_visit_length_minutes', 'avg_visits_per_unq_visitor'])
df.head(10)

,date,total_visitors,unq_visitors,avg_visit_length_minutes,avg_visits_per_unq_visitor
0,2023-12-22,18,18,156.241667,1.000000
1,2023-12-23,24,24,155.009722,1.000000
2,2023-12-24,17,16,148.649020,1.062500
3,2023-12-25,14,12,145.578572,1.166667
4,2023-12-26,23,23,123.883333,1.000000
5,2023-12-27,19,19,164.612281,1.000000
6,2023-12-28,25,23,108.116667,1.086957
7,2023-12-29,6,6,113.191667,1.000000


In [157]:
daily_visitors

[array(['82.jpg', '28.jpg', '11.jpg', '40.jpg', '25.jpg', '72.jpg',
        '74.jpg', '85.jpg', '6.jpg', '23.jpg', '54.jpg', '9.jpg', '46.jpg',
        '19.jpg', '18.jpg', '34.jpg', '75.jpg', '36.jpg', '83.jpg',
        '80.jpg', '70.jpg', '24.jpg', '21.jpg', '.DS_Store', '30.jpg',
        '50.jpg', '68.jpg'], dtype='<U9'),
 array(['8.jpg', '13.jpg', '73.jpg', '9.jpg', '26.jpg', '61.jpg', '6.jpg',
        '33.jpg', '38.jpg', '24.jpg', '83.jpg', '40.jpg', '27.jpg',
        '1.jpg', '21.jpg', '30.jpg'], dtype='<U9'),
 array(['47.jpg', '50.jpg', '75.jpg', '69.jpg', '33.jpg', '61.jpg',
        '28.jpg', '72.jpg', '66.jpg', '80.jpg', '85.jpg', '11.jpg',
        '46.jpg', '36.jpg', '.DS_Store', '67.jpg', '12.jpg', '9.jpg',
        '18.jpg', '24.jpg', '37.jpg', '71.jpg', '60.jpg', '84.jpg'],
       dtype='<U9'),
 array(['75.jpg', '23.jpg', '38.jpg', '5.jpg', '82.jpg', '24.jpg',
        '18.jpg', '28.jpg', '.DS_Store', '2.jpg', '6.jpg'], dtype='<U9'),
 array(['84.jpg', '24.jpg', '27.jpg', '5.j